In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

# Exploratory data analysis

Read the database file.

In [ ]:
df = pd.read_excel('Data/BBDD_Hospitalización.xlsx')
df.head()

In [ ]:
df.shape

Check columns information.

In [ ]:
df.info()

In [ ]:
df.describe()

Check how many nulls values are in dataframe.

In [ ]:
df.isnull().sum()

Print the content of each column.

In [ ]:
columns = df.columns.to_list()
columns.pop(0)
columns

for column in columns:
    print(f'{column} = \n {df[column].value_counts()}')
    print('\n')

# Data preparation

Replace string of numeric column.

In [ ]:
df['NUMERO DE DIAS POST BIOPSIA EN QUE SE PRESENTA LA COMPLICACIÓN INFECCIOSA'].replace('NO', 0, inplace=True)
df['NUMERO DE DIAS POST BIOPSIA EN QUE SE PRESENTA LA COMPLICACIÓN INFECCIOSA'].value_counts()

Delete wrong characters

In [ ]:
for column in columns:
    if df[column].dtype == 'object':
        df[column] = df[column].str.strip()

Delete row if all the fields are `NaN`, otherwise not.

In [ ]:
df.dropna(how='all', inplace=True)

Check how many nulls values are in dataframe.

In [ ]:
df.isnull().sum()

In [ ]:
df['ANTIBIOTICO UTILIAZADO EN LA PROFILAXIS'].replace('FLUOROQUINOLONA_AMINOGLICÓSIDO', 'FLUOROQUINOLONA_AMINOGLICOSIDO', inplace=True)


Check for outlier in column `EDAD`.

In [ ]:
df_aux = df[df['EDAD'] > 100]
df_aux

In [ ]:
df_aux = df[df['EDAD'] < 0]
df_aux

Delete row which contains wrong information

In [ ]:
df.drop([161, 181], inplace=True)


Check for duplicates and delete them.

In [ ]:
df.duplicated().sum()

In [ ]:
df[df.duplicated()]

In [ ]:
df.drop_duplicates()

In [ ]:
df.isnull().sum()

Fill missing values with median.

In [ ]:
df['PSA'].fillna(value=df['PSA'].median(), inplace=True)

Delete missing values.

In [ ]:
df.dropna(axis=0, inplace=True)

Reset indexes.

In [ ]:
df.reset_index(drop=True)

Change columns names.

In [ ]:
columns_name_list = df.columns.to_list()
columns_name_list
columns_name_dict = {column:index for index,column in enumerate(columns_name_list)}
columns_name_dict

In [ ]:
columns_name_dict = {'EDAD': 'edad',
                    'DIABETES': 'diabetes',
                    'HOSPITALIZACIÓN ULTIMO MES': 'hum',
                    'PSA': 'psa',
                    'BIOPSIAS PREVIAS': 'bp',
                    'VOLUMEN PROSTATICO': 'vp',
                    'ANTIBIOTICO UTILIAZADO EN LA PROFILAXIS': 'aup',
                    'NUMERO DE MUESTRAS TOMADAS': 'nmt',
                    'CUP': 'cup',
                    'ENF. CRONICA PULMONAR OBSTRUCTIVA': 'ecpo',
                    'BIOPSIA': 'biopsia',
                    'NUMERO DE DIAS POST BIOPSIA EN QUE SE PRESENTA LA COMPLICACIÓN INFECCIOSA': 'ndpb_pci',
                    'FIEBRE': 'fiebre',
                    'ITU': 'itu',
                    'TIPO DE CULTIVO': 'tipo_cultivo',
                    'AGENTE AISLADO': 'agente_aislado',
                    'PATRON DE RESISTENCIA': 'patron_resistencia',
                    'HOSPITALIZACION': 'hospitalizacion',
                    'DIAS HOSPITALIZACION MQ': 'dias_hosp_mq',
                    'DIAS HOSPITALIZACIÓN UPC': 'dias_hosp_upc'}
df.rename(columns=columns_name_dict, inplace=True)

Check columns values again.

In [ ]:
columns = df.columns.to_list()
columns.pop(0)
columns

for column in columns:
    if df[column].dtype == 'object':
        df[column] = df[column].str.strip()
    print(f'{column} = \n {df[column].value_counts()}')
    print('\n')

Change SI/NO values for 1/0

In [ ]:
yes_no_list_column = {'SI':1, 'NO':0}
dict_si_no = ['diabetes', 'hum', 'bp', 'vp', 'cup', 'fiebre', 'itu', 'hospitalizacion']
for column in dict_si_no:
    df.replace({column: yes_no_list_column}, inplace=True)
df.head()

Check dataframe columns types.

In [ ]:
df.info()

Replace qualitative values for quantitative ones.

In [ ]:
def replace_function(dataframe, column):
    """Replace qualitative values for quantitative ones.

    Args:
        dataframe (DataFrame): Dataframe.
        column (str): Column name
    """
    var_name = dataframe[column].unique().tolist()
    var_name_dict = {var:index for index, var in enumerate(var_name)}
    dataframe.replace({column: var_name_dict}, inplace=True)
    print(column)
    print(var_name_dict)

In [ ]:
replace_function(df, 'biopsia')
replace_function(df, 'ecpo')
replace_function(df, 'tipo_cultivo')
replace_function(df, 'agente_aislado')
replace_function(df, 'patron_resistencia')
replace_function(df, 'aup')
df.head()

# Modeling and evaluation

In [ ]:
# columns = df.columns.to_list()
columns = ['diabetes', 'psa', 'bp', 'vp', 'aup', 'nmt', 'biopsia', 'fiebre', 'hospitalizacion', 'dias_hosp_mq']

Heat map to check correlation variables

In [ ]:
df_aux = df[columns]
df_aux.head()

In [ ]:
df_aux = df[columns]
corr = df_aux.corr()
plt.figure(figsize=(15,15))
sb.heatmap(corr, cbar = True,  square = True, annot=True, fmt= '.2f', annot_kws={'size': 12},
            xticklabels= df_aux.columns, 
            yticklabels= df_aux.columns,
            cmap= 'coolwarm')
plt.show()

In [ ]:
sb.pairplot(df_aux, hue = 'hospitalizacion')
plt.show()

In [ ]:
df_aux.describe()

In [ ]:
columns_list = df_aux.columns.to_list()
columns_list.remove('hospitalizacion')
y = df['hospitalizacion'].values

In [ ]:
for column in columns_list:
    sb.scatterplot(df_aux[column].values, y, )
    plt.show()
